<a href="https://colab.research.google.com/github/Benny-rgb-star/DATASPRINT-DSIG-10/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# If running in Colab, you can uncomment:
!pip -q install fastapi uvicorn jinja2 passlib[bcrypt] sqlalchemy python-multipart apscheduler

import os, io, csv, json, math, re, sys, time, socket, shutil, stat, subprocess, threading
from datetime import datetime, date, timedelta, timezone
from zoneinfo import ZoneInfo
from typing import Optional, List, Dict, Any

from fastapi import FastAPI, Request, Form, Depends, UploadFile, File, HTTPException, status, Query
from fastapi.responses import HTMLResponse, RedirectResponse, JSONResponse, StreamingResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from starlette.middleware.sessions import SessionMiddleware
from fastapi.middleware.cors import CORSMiddleware

from passlib.context import CryptContext
from sqlalchemy import (create_engine, Column, Integer, String, DateTime, Boolean,
                        ForeignKey, UniqueConstraint, Date, Text)
from sqlalchemy.orm import sessionmaker, declarative_base, relationship, Session as OrmSession

from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.cron import CronTrigger

# ---------------------------
# Config
# ---------------------------
ADMIN_PASSWORD = os.environ.get("ADMIN_PASSWORD", "hackathon-Admin#2025")
SECRET_KEY = os.environ.get("SESSION_SECRET", "super-secret-session-key-change-me")
DB_URL = "sqlite:///./wellness.db"
IST = ZoneInfo("Asia/Kolkata")

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

app = FastAPI(title="Corporate Wellness API")
app.add_middleware(SessionMiddleware, secret_key=SECRET_KEY)

# CORS for your hosted frontend (adjust origin as needed; for quick tests you can use ["*"])
app.add_middleware(
    CORSMiddleware,
    allow_origins=["https://dsig-10-gamified-fit-xdq0.bolt.host"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

os.makedirs("templates", exist_ok=True)
os.makedirs("static", exist_ok=True)
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# ---------------------------
# DB
# ---------------------------
engine = create_engine(DB_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

def utcnow() -> datetime:
    return datetime.now(timezone.utc)

def today_utc() -> date:
    return utcnow().date()

# ---------------------------
# Models
# ---------------------------
class Team(Base):
    __tablename__ = "teams"
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True, nullable=False)
    created_at = Column(DateTime, default=utcnow)
    users = relationship("User", back_populates="team")

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    emp_id = Column(String, unique=True, index=True, nullable=False)
    name = Column(String, nullable=False)
    email = Column(String, nullable=True)
    email_verified = Column(Boolean, default=True)  # demo
    password_hash = Column(String, nullable=False)
    is_admin = Column(Boolean, default=False)
    team_id = Column(Integer, ForeignKey("teams.id"), nullable=True)
    is_active = Column(Boolean, default=True)
    wearable_provider = Column(String, nullable=True)
    wearable_token = Column(String, nullable=True)
    created_at = Column(DateTime, default=utcnow)

    team = relationship("Team", back_populates="users")
    enrollments = relationship("Enrollment", back_populates="user")
    prefs = relationship("UserPrefs", back_populates="user", uselist=False)

class UserPrefs(Base):
    __tablename__ = "user_prefs"
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey("users.id"), unique=True, nullable=False)
    daily_step_goal = Column(Integer, default=8000)
    created_at = Column(DateTime, default=utcnow)

    user = relationship("User", back_populates="prefs")

class Challenge(Base):
    __tablename__ = "challenges"
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    metric = Column(String, nullable=False)  # 'steps' | 'calories'
    start_at = Column(DateTime, nullable=False)  # UTC
    end_at = Column(DateTime, nullable=False)    # UTC
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=utcnow)

    enrollments = relationship("Enrollment", back_populates="challenge")

class Enrollment(Base):
    __tablename__ = "enrollments"
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey("users.id"), nullable=False)
    challenge_id = Column(Integer, ForeignKey("challenges.id"), nullable=False)
    enrolled_at = Column(DateTime, default=utcnow)

    user = relationship("User", back_populates="enrollments")
    challenge = relationship("Challenge", back_populates="enrollments")

    __table_args__ = (UniqueConstraint("user_id","challenge_id", name="uq_enroll_user_ch"),)

class ActivityLog(Base):
    __tablename__ = "activity_logs"
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey("users.id"), nullable=False)
    date_utc = Column(Date, nullable=False)
    steps = Column(Integer, default=0)
    calories = Column(Integer, default=0)
    source = Column(String, default="manual")
    flagged = Column(Boolean, default=False)
    flag_reason = Column(String, nullable=True)
    created_at = Column(DateTime, default=utcnow)  # first insert time
    updated_at = Column(DateTime, default=utcnow)

    __table_args__ = (UniqueConstraint("user_id","date_utc", name="uq_activity_user_day"),)

class LeaderboardSnapshot(Base):
    __tablename__ = "leaderboard_snapshots"
    id = Column(Integer, primary_key=True)
    challenge_id = Column(Integer, ForeignKey("challenges.id"), nullable=False)
    run_at = Column(DateTime, default=utcnow)
    scope = Column(String, default="daily")  # 'daily' | 'final'
    payload = Column(Text, nullable=False)

    challenge = relationship("Challenge")

Base.metadata.create_all(bind=engine)

# ---------------------------
# Helpers
# ---------------------------
MAX_DAILY_STEPS = 60000  # cap
SPIKE_RATIO = 3.5        # >3.5x last-7d avg steps => suspicious

def hash_password(plain: str) -> str:
    return pwd_context.hash(plain)

def verify_password(plain: str, hashed: str) -> bool:
    return pwd_context.verify(plain, hashed)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def parse_iso_utc(s: str) -> datetime:
    s = s.strip()
    if s.endswith("Z"):
        s = s[:-1] + "+00:00"
    dt = datetime.fromisoformat(s)
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

def require_admin_session(request: Request):
    if request.session.get("role") != "admin":
        raise HTTPException(status_code=403, detail="Admin access required.")

def require_employee_session(request: Request):
    if request.session.get("role") != "employee":
        raise HTTPException(status_code=403, detail="Employee access required.")

def get_or_create_user_prefs(db: OrmSession, user_id: int) -> UserPrefs:
    prefs = db.query(UserPrefs).filter(UserPrefs.user_id == user_id).first()
    if not prefs:
        prefs = UserPrefs(user_id=user_id, daily_step_goal=8000)
        db.add(prefs); db.commit()
    return prefs

def last_7_avg_steps(db: OrmSession, user_id: int, ref_day: date) -> float:
    start = ref_day - timedelta(days=7)
    rows = db.query(ActivityLog).filter(
        ActivityLog.user_id == user_id,
        ActivityLog.date_utc >= start,
        ActivityLog.date_utc < ref_day
    ).all()
    vals = [r.steps for r in rows if not r.flagged]
    return (sum(vals) / len(vals)) if vals else 0.0

def anti_cheat_flag(db: OrmSession, user_id: int, day: date, steps: int) -> tuple[bool,str]:
    if steps > MAX_DAILY_STEPS:
        return True, f"over_cap_{MAX_DAILY_STEPS}"
    avg7 = last_7_avg_steps(db, user_id, day)
    if avg7 > 0 and steps > SPIKE_RATIO * avg7 and steps >= 15000:
        return True, f"spike_gt_{SPIKE_RATIO}x_avg7"
    return False, ""

def clamp_activity(steps: int, calories: int) -> tuple[int,int]:
    steps = max(0, min(MAX_DAILY_STEPS, int(steps or 0)))
    calories = max(0, int(calories or 0))
    return steps, calories

def aggregate_user_metric(db: OrmSession, user_id: int, start_at: datetime, end_at: datetime, metric: str, exclude_flagged=True) -> int:
    q = db.query(ActivityLog).filter(
        ActivityLog.user_id==user_id,
        ActivityLog.date_utc >= start_at.date(),
        ActivityLog.date_utc <= end_at.date()
    )
    total = 0
    for row in q:
        if exclude_flagged and row.flagged:
            continue
        total += row.steps if metric == "steps" else row.calories
    return total

def earliest_reach_timestamp(db: OrmSession, user_id: int, start_at: datetime, end_at: datetime, metric: str, target_total: int) -> Optional[datetime]:
    """Earliest time cumulative total met target_total (approx using per-day created_at)."""
    if target_total <= 0:
        return None
    rows = db.query(ActivityLog).filter(
        ActivityLog.user_id==user_id,
        ActivityLog.date_utc >= start_at.date(),
        ActivityLog.date_utc <= end_at.date()
    ).order_by(ActivityLog.date_utc.asc(), ActivityLog.created_at.asc()).all()
    cum = 0
    for r in rows:
        if r.flagged:
            continue
        add = r.steps if metric=="steps" else r.calories
        cum += add
        if cum >= target_total:
            return r.created_at
    return None

def compute_streaks(db: OrmSession, user_id: int, goal: int, lookback_days: int = 120) -> dict:
    today = today_utc()
    days = [(today - timedelta(days=(lookback_days - 1 - i))) for i in range(lookback_days)]
    hits = []
    for d in days:
        r = db.query(ActivityLog).filter(ActivityLog.user_id==user_id, ActivityLog.date_utc==d).first()
        steps = (r.steps if r and not r.flagged else 0)
        hits.append(steps >= goal)
    longest = cur = 0
    for ok in hits:
        if ok: cur += 1; longest = max(longest, cur)
        else: cur = 0
    current = 0
    for ok in reversed(hits):
        if ok: current += 1
        else: break
    return {"current_streak_days": current, "longest_streak_days": longest}

def enrollment_allowed(ch: Challenge) -> bool:
    now = utcnow()
    return ch.is_active and now <= ch.end_at

def compute_leaderboard(db: OrmSession, ch: Challenge) -> list[dict]:
    entries = []
    now = utcnow()
    cutoff = min(ch.end_at, now)
    ens = db.query(Enrollment).filter(Enrollment.challenge_id == ch.id).all()
    for en in ens:
        start = max(ch.start_at, en.enrolled_at)  # late enrollment rule
        total = aggregate_user_metric(db, en.user_id, start, cutoff, ch.metric, exclude_flagged=True)
        prefs = get_or_create_user_prefs(db, en.user_id)
        s = compute_streaks(db, en.user_id, prefs.daily_step_goal, lookback_days=120)
        first_ts = earliest_reach_timestamp(db, en.user_id, start, cutoff, ch.metric, total)
        entries.append({
            "emp_id": en.user.emp_id,
            "name": en.user.name,
            "team": (en.user.team.name if en.user.team else "—"),
            "total": total,
            "streak": s["current_streak_days"],
            "first_reach_ts": (first_ts.isoformat().replace("+00:00","Z") if first_ts else None)
        })
    # Sort: total desc, earliest reach asc, name/emp_id fallback
    def sort_key(e):
        ts = e["first_reach_ts"]
        ts_tuple = (0, ts) if ts else (1, "")
        return (-e["total"], ts_tuple, e["name"].lower(), e["emp_id"].lower())
    entries.sort(key=sort_key)
    for i, e in enumerate(entries, start=1):
        e["rank"] = i
    return entries

def store_snapshot(db: OrmSession, challenge_id: int, scope: str, entries: list[dict], metric: str):
    payload = {
        "metric": metric,
        "entries": entries,
        "when": utcnow().isoformat().replace("+00:00","Z")
    }
    snap = LeaderboardSnapshot(challenge_id=challenge_id, scope=scope, payload=json.dumps(payload))
    db.add(snap); db.commit()

def recompute_all_leaderboards(scope: str = "daily"):
    db = SessionLocal()
    try:
        now = utcnow()
        for ch in db.query(Challenge).all():
            if ch.is_active and now > ch.end_at:
                entries = compute_leaderboard(db, ch)
                store_snapshot(db, ch.id, "final", entries, ch.metric)
                ch.is_active = False
                db.commit()
            if ch.is_active and ch.start_at <= now:
                entries = compute_leaderboard(db, ch)
                store_snapshot(db, ch.id, scope, entries, ch.metric)
    finally:
        db.close()

# ---------------------------
# CSS & Templates
# ---------------------------
base_css = """
:root{ --bg:#0e1117; --card:#161b22; --muted:#8b949e; --text:#e6edf3; --accent:#2f81f7; }
*{ box-sizing:border-box; } body{ margin:0; font-family:ui-sans-serif,system-ui,-apple-system,Segoe UI,Roboto; background:var(--bg); color:var(--text); }
a{ color:var(--accent); text-decoration:none; } .container{ max-width:1120px; margin:40px auto; padding:0 16px; }
.card{ background:var(--card); border:1px solid #30363d; border-radius:16px; padding:24px; box-shadow: 0 8px 24px rgba(0,0,0,0.25); }
h1,h2{ margin:0 0 12px 0; } .btn{ display:inline-block; padding:10px 16px; border-radius:10px; background:var(--accent); color:white; border:none; cursor:pointer; font-weight:600; }
.btn.secondary{ background:#30363d; color:var(--text); } .row{ display:flex; gap:12px; flex-wrap:wrap; } .input{ width:100%; padding:10px 12px; border-radius:10px; border:1px solid #30363d; background:#0b0f14; color:var(--text); }
.grid{ display:grid; grid-template-columns: 260px 1fr; gap:16px; } .sidebar{ background:#0b0f14; border:1px solid #30363d; border-radius:16px; padding:16px; }
.sidebar a{ display:block; padding:10px 12px; border-radius:8px; color:var(--text); } .sidebar a:hover{ background:#11161d; }
.badge{ display:inline-block; padding:4px 8px; border-radius:999px; background:#232a33; color:var(--muted); font-size:12px; }
.flash{ background:#10233f; border:1px solid #1f4272; padding:10px 12px; border-radius:8px; margin-bottom:12px; color:#bcd2f7; }
.table{ width:100%; border-collapse:collapse; margin-top:12px; } .table th,.table td{ padding:8px 10px; border-bottom:1px solid #2a3038; text-align:left; }
.small{ color:var(--muted); font-size:12px; }
"""
with open("static/styles.css","w") as f: f.write(base_css)

layout_html = """
<!doctype html><html><head><meta charset="utf-8" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<title>{{ title or "Wellness API" }}</title>
<link rel="stylesheet" href="/static/styles.css"></head><body>
{% if flash %}<div class="container"><div class="flash">{{ flash }}</div></div>{% endif %}
{% block content %}{% endblock %}
</body></html>
"""
index_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="card">
<h1>Corporate Wellness</h1>
<p class="badge">UTC engine • Daily refresh at 09:29 IST</p>
<p class="small">Admin or Employee?</p>
<div class="row" style="margin-top:16px">
  <a class="btn" href="/admin/login">Admin</a>
  <a class="btn secondary" href="/employee">Employee</a>
</div></div></div>
{% endblock %}
"""
admin_login_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="card">
<h2>Admin Login</h2>
<form method="post" action="/admin/login">
  <label>Password</label>
  <input class="input" type="password" name="password" required />
  <div class="row" style="margin-top:12px">
    <button class="btn" type="submit">Login</button>
    <a class="btn secondary" href="/">Back</a>
  </div>
</form>
<p class="small" style="margin-top:8px">Default: <code>hackathon-Admin#2025</code></p>
</div></div>
{% endblock %}
"""
admin_dash_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="grid">
<div class="sidebar">
  <strong>Admin Menu</strong>
  <a href="/admin/dashboard">Dashboard</a>
  <a href="/admin/challenges">Challenges</a>
  <a href="/admin/challenges/new">Create Challenge</a>
  <a href="/admin/leaderboard/run">Recompute Leaderboards</a>
  <a href="/admin/hr">HR Import</a>
  <a href="/admin/users">Users</a>
  <a href="/logout">Logout</a>
</div>
<div class="card">
  <h2>Challenge Progress</h2>
  {% if challenges %}
  <table class="table">
    <thead><tr><th>Name</th><th>Metric</th><th>Start (UTC)</th><th>End (UTC)</th><th>Active</th><th>Enrollments</th></tr></thead>
    <tbody>
      {% for c in challenges %}
      <tr>
        <td>{{ c.name }}</td>
        <td>{{ c.metric }}</td>
        <td>{{ c.start_at }}</td>
        <td>{{ c.end_at }}</td>
        <td>{{ "Yes" if c.is_active else "No" }}</td>
        <td>{{ c.enroll_count }}</td>
      </tr>
      {% endfor %}
    </tbody>
  </table>
  {% else %}<p>No challenges yet. Create one!</p>{% endif %}

  {% if last_snapshot %}
    <h3 style="margin-top:16px">Last Leaderboard Snapshot</h3>
    <p class="badge">{{ last_snapshot["when"] }}</p>
    <div class="row" style="margin:6px 0">
      <a class="btn secondary" href="/admin/leaderboard/export?scope={{ last_snapshot['scope'] }}&challenge_id={{ last_snapshot['challenge_id'] }}">Export CSV</a>
    </div>
    <table class="table">
      <thead><tr><th>Rank</th><th>Name</th><th>Emp ID</th><th>Team</th><th>Total</th><th>Streak</th></tr></thead>
      <tbody>
      {% for e in last_snapshot["entries"][:10] %}
        <tr>
          <td>{{ e.rank }}</td>
          <td>{{ e.name }}</td>
          <td>{{ e.emp_id }}</td>
          <td>{{ e.team }}</td>
          <td>{{ e.total }}</td>
          <td><span class="badge">🔥 {{ e.streak }}d</span></td>
        </tr>
      {% endfor %}
      </tbody>
    </table>
  {% endif %}
</div></div></div>
{% endblock %}
"""
admin_new_challenge_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="grid">
<div class="sidebar">
  <strong>Admin Menu</strong>
  <a href="/admin/dashboard">Dashboard</a>
  <a href="/admin/challenges">Challenges</a>
  <a href="/logout">Logout</a>
</div>
<div class="card">
  <h2>Create Challenge</h2>
  <form method="post" action="/admin/challenges/new">
    <label>Name</label><input class="input" name="name" required placeholder="e.g., September Steps Sprint" />
    <label style="margin-top:8px">Metric</label>
    <select class="input" name="metric">
      <option value="steps">steps</option>
      <option value="calories">calories</option>
    </select>
    <label style="margin-top:8px">Start (UTC, ISO8601)</label>
    <input class="input" name="start_at" required placeholder="2025-09-01T00:00:00Z" />
    <label style="margin-top:8px">End (UTC, ISO8601)</label>
    <input class="input" name="end_at" required placeholder="2025-09-30T23:59:59Z" />
    <div class="row" style="margin-top:12px">
      <button class="btn" type="submit">Create</button>
      <a class="btn secondary" href="/admin/challenges">Cancel</a>
    </div>
  </form>
</div></div></div>
{% endblock %}
"""
admin_challenges_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="grid">
<div class="sidebar">
  <strong>Admin Menu</strong>
  <a href="/admin/dashboard">Dashboard</a>
  <a href="/admin/challenges/new">Create Challenge</a>
  <a href="/admin/leaderboard/run">Recompute Leaderboards</a>
  <a href="/logout">Logout</a>
</div>
<div class="card">
  <h2>Challenges</h2>
  {% if items %}
  <table class="table">
    <thead><tr><th>Name</th><th>Metric</th><th>Start</th><th>End</th><th>Active</th><th>Actions</th></tr></thead>
    <tbody>
      {% for c in items %}
      <tr>
        <td>{{ c.name }}</td><td>{{ c.metric }}</td><td>{{ c.start_at }}</td><td>{{ c.end_at }}</td><td>{{ "Yes" if c.is_active else "No" }}</td>
        <td>
          {% if c.is_active %}<a class="btn secondary" href="/admin/challenges/{{ c.id }}/close">Close</a>{% endif %}
          <a class="btn secondary" href="/admin/challenges/{{ c.id }}/delete">Delete</a>
        </td>
      </tr>
      {% endfor %}
    </tbody>
  </table>
  {% else %}<p>No challenges yet.</p>{% endif %}
</div></div></div>
{% endblock %}
"""
admin_hr_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="grid">
<div class="sidebar">
  <strong>Admin Menu</strong>
  <a href="/admin/dashboard">Dashboard</a>
  <a href="/admin/users">Users</a>
  <a href="/logout">Logout</a>
</div>
<div class="card">
  <h2>HR CSV Import</h2>
  <p class="small">CSV headers: <code>name,email,emp_id,team</code></p>
  <form method="post" action="/admin/hr" enctype="multipart/form-data">
    <input class="input" type="file" name="file" accept=".csv" required />
    <div class="row" style="margin-top:12px">
      <button class="btn" type="submit">Upload</button>
      <a class="btn secondary" href="/admin/dashboard">Back</a>
    </div>
  </form>
</div></div></div>
{% endblock %}
"""
admin_users_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="grid">
<div class="sidebar">
  <strong>Admin Menu</strong>
  <a href="/admin/dashboard">Dashboard</a>
  <a href="/admin/hr">HR Import</a>
  <a href="/logout">Logout</a>
</div>
<div class="card">
  <h2>Users</h2>
  <table class="table">
    <thead><tr><th>Name</th><th>Emp ID</th><th>Email</th><th>Team</th><th>Active</th><th>Actions</th></tr></thead>
    <tbody>
      {% for u in users %}
      <tr>
        <td>{{ u.name }}</td>
        <td>{{ u.emp_id }}</td>
        <td>{{ u.email or "—" }}</td>
        <td>{{ u.team or "—" }}</td>
        <td>{{ "Yes" if u.is_active else "No" }}</td>
        <td>
          <a class="btn secondary" href="/admin/users/{{ u.emp_id }}/disable">Toggle Active</a>
          <a class="btn secondary" href="/admin/users/{{ u.emp_id }}/reset">Reset Password</a>
        </td>
      </tr>
      {% endfor %}
    </tbody>
  </table>
</div></div></div>
{% endblock %}
"""
employee_home_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="card">
  <h2>Employee Portal</h2>
  <div class="row" style="margin-top:12px">
    <a class="btn" href="/employee/signup">Sign Up (New User)</a>
    <a class="btn secondary" href="/employee/login">Sign In</a>
    <a class="btn secondary" href="/employee/forgot">Forgot Password</a>
    <a class="btn secondary" href="/">Back</a>
  </div>
</div></div>
{% endblock %}
"""
employee_login_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="card">
  <h2>Employee Sign In</h2>
  <form method="post" action="/employee/login">
    <label>Employee ID</label><input class="input" type="text" name="emp_id" required />
    <label style="margin-top:8px">Password</label><input class="input" type="password" name="password" required />
    <div class="row" style="margin-top:12px">
      <button class="btn" type="submit">Sign In</button><a class="btn secondary" href="/employee">Back</a>
    </div>
  </form>
</div></div>
{% endblock %}
"""
employee_signup_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="card">
  <h2>Sign Up (New User)</h2>
  <form method="post" action="/employee/signup">
    <label>Name</label><input class="input" type="text" name="name" required />
    <label style="margin-top:8px">Employee ID</label><input class="input" type="text" name="emp_id" required />
    <label style="margin-top:8px">Email</label><input class="input" type="email" name="email" required />
    <label style="margin-top:8px">Password</label><input class="input" type="password" name="password" required />
    <label style="margin-top:8px">Daily Step Goal</label>
    <input class="input" type="number" min="0" step="100" name="daily_step_goal" value="8000" required />
    <div class="row" style="margin-top:12px">
      <button class="btn" type="submit">Create Account</button><a class="btn secondary" href="/employee">Back</a>
    </div>
  </form>
</div></div>
{% endblock %}
"""
employee_dash_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="grid">
<div class="sidebar">
  <strong>Employee Menu</strong>
  <a href="/employee/dashboard">My Dashboard</a>
  <a href="/employee/goals">My Goal</a>
  <a href="/employee/wearable">Connect Wearable</a>
  <a href="/employee/challenges">Challenges</a>
  <a href="/logout">Logout</a>
</div>
<div class="card">
  <h2>Welcome, {{ user.name }}</h2>
  <p class="badge">Emp ID: {{ user.emp_id }}</p>
  <p class="badge">Email: {{ user.email or "—" }} • {{ "Verified" if user.email_verified else "Not Verified" }}</p>

  <h3 style="margin-top:12px">Today’s Stats (UTC {{ today }})</h3>
  <table class="table"><tbody>
    <tr><th>Steps</th><td>{{ today_stats.steps }}</td></tr>
    <tr><th>Calories</th><td>{{ today_stats.calories }}</td></tr>
  </tbody></table>

  <h3 style="margin-top:12px">Personal Space</h3>
  <table class="table"><tbody>
    <tr><th>Daily Step Goal</th><td>{{ personal.goal }}</td></tr>
    <tr><th>Avg Steps (7d)</th><td>{{ personal.avg_steps_7 }}</td></tr>
    <tr><th>Avg Steps (30d)</th><td>{{ personal.avg_steps_30 }}</td></tr>
    <tr><th>Avg Calories (7d)</th><td>{{ personal.avg_cal_7 }}</td></tr>
    <tr><th>Avg Calories (30d)</th><td>{{ personal.avg_cal_30 }}</td></tr>
    <tr><th>Current Streak (days)</th><td><span class="badge">🔥 {{ personal.current_streak }}</span></td></tr>
    <tr><th>Longest Streak (days)</th><td>{{ personal.longest_streak }}</td></tr>
  </tbody></table>
  <div class="row" style="margin-top:8px">
    <a class="btn" href="/employee/goals">Edit Daily Goal</a>
  </div>
</div></div></div>
{% endblock %}
"""
employee_goals_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="grid">
<div class="sidebar">
  <strong>Employee Menu</strong>
  <a href="/employee/dashboard">My Dashboard</a>
  <a href="/employee/wearable">Connect Wearable</a>
  <a href="/employee/challenges">Challenges</a>
  <a href="/logout">Logout</a>
</div>
<div class="card">
  <h2>My Daily Goal</h2>
  <form method="post" action="/employee/goals">
    <label>Daily Step Goal</label>
    <input class="input" type="number" min="0" step="100" name="daily_step_goal" value="{{ daily_step_goal }}" required />
    <div class="row" style="margin-top:12px">
      <button class="btn" type="submit">Save</button>
      <a class="btn secondary" href="/employee/dashboard">Back</a>
    </div>
  </form>
  <p class="small">Streak counts days you meet/exceed this goal.</p>
</div></div></div>
{% endblock %}
"""
employee_wearable_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="grid">
<div class="sidebar">
  <strong>Employee Menu</strong>
  <a href="/employee/dashboard">My Dashboard</a>
  <a href="/employee/challenges">Challenges</a>
  <a href="/logout">Logout</a>
</div>
<div class="card">
  <h2>Connect Wearable (Mock)</h2>
  <form method="post" action="/employee/wearable">
    <label>Provider</label>
    <select class="input" name="provider">
      <option value="googlefit">Google Fit</option>
      <option value="fitbit">Fitbit</option>
      <option value="apple">Apple Health</option>
    </select>
    <label style="margin-top:8px">Mock Token</label>
    <input class="input" name="token" placeholder="e.g., demo-token-123" />
    <div class="row" style="margin-top:12px">
      <button class="btn" type="submit">Connect</button><a class="btn secondary" href="/employee/dashboard">Back</a>
    </div>
  </form>
  <p class="small">Demo stores a mock token. Real OAuth can be added later.</p>
</div></div></div>
{% endblock %}
"""
employee_challenges_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="grid">
<div class="sidebar">
  <strong>Employee Menu</strong>
  <a href="/employee/dashboard">My Dashboard</a>
  <a href="/employee/wearable">Connect Wearable</a>
  <a href="/logout">Logout</a>
</div>
<div class="card">
  <h2>Challenges</h2>
  {% if open_challenges %}
  <h3>Open for Enrollment</h3>
  <table class="table">
    <thead><tr><th>Name</th><th>Metric</th><th>Start</th><th>End</th><th>Action</th></tr></thead>
    <tbody>
      {% for c in open_challenges %}
      <tr>
        <td>{{ c.name }}</td><td>{{ c.metric }}</td><td>{{ c.start_at }}</td><td>{{ c.end_at }}</td>
        <td><a class="btn" href="/employee/challenges/enroll?challenge_id={{ c.id }}">Enroll</a></td>
      </tr>
      {% endfor %}
    </tbody>
  </table>
  {% else %}<p>No open challenges right now.</p>{% endif %}

  {% if my_challenges %}
  <h3 style="margin-top:16px">My Enrollments</h3>
  <table class="table">
    <thead><tr><th>Name</th><th>Metric</th><th>Start</th><th>End</th></tr></thead>
    <tbody>
      {% for c in my_challenges %}
      <tr><td>{{ c.name }}</td><td>{{ c.metric }}</td><td>{{ c.start_at }}</td><td>{{ c.end_at }}</td></tr>
      {% endfor %}
    </tbody>
  </table>
  {% endif %}
</div></div></div>
{% endblock %}
"""
employee_forgot_html = """
{% extends "layout.html" %}{% block content %}
<div class="container"><div class="card">
  <h2>Forgot Password (Demo)</h2>
  <form method="post" action="/employee/forgot">
    <label>Employee ID</label><input class="input" type="text" name="emp_id" required />
    <label style="margin-top:8px">New Password</label><input class="input" type="password" name="new_password" required />
    <label style="margin-top:8px">Mock Verification Code</label><input class="input" type="text" name="code" placeholder="enter 000000" required />
    <div class="row" style="margin-top:12px">
      <button class="btn" type="submit">Reset</button>
      <a class="btn secondary" href="/employee">Back</a>
    </div>
  </form>
  <p class="small">Demo only — code is always <code>000000</code>.</p>
</div></div>
{% endblock %}
"""

# write templates
with open("templates/layout.html","w") as f: f.write(layout_html)
with open("templates/index.html","w") as f: f.write(index_html)
with open("templates/admin_login.html","w") as f: f.write(admin_login_html)
with open("templates/admin_dashboard.html","w") as f: f.write(admin_dash_html)
with open("templates/admin_new_challenge.html","w") as f: f.write(admin_new_challenge_html)
with open("templates/admin_challenges.html","w") as f: f.write(admin_challenges_html)
with open("templates/admin_hr.html","w") as f: f.write(admin_hr_html)
with open("templates/admin_users.html","w") as f: f.write(admin_users_html)
with open("templates/employee_home.html","w") as f: f.write(employee_home_html)
with open("templates/employee_login.html","w") as f: f.write(employee_login_html)
with open("templates/employee_signup.html","w") as f: f.write(employee_signup_html)
with open("templates/employee_dashboard.html","w") as f: f.write(employee_dash_html)
with open("templates/employee_goals.html","w") as f: f.write(employee_goals_html)
with open("templates/employee_wearable.html","w") as f: f.write(employee_wearable_html)
with open("templates/employee_challenges.html","w") as f: f.write(employee_challenges_html)
with open("templates/employee_forgot.html","w") as f: f.write(employee_forgot_html)

# ---------------------------
# Routes: Common
# ---------------------------
@app.get("/", response_class=HTMLResponse)
def home(request: Request):
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("index.html", {"request": request, "title":"Home", "flash": flash})

# ---------------------------
# Admin
# ---------------------------
@app.get("/admin/login", response_class=HTMLResponse)
def admin_login_form(request: Request):
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("admin_login.html", {"request": request, "title":"Admin Login", "flash": flash})

@app.post("/admin/login")
def admin_login(request: Request, password: str = Form(...)):
    if password != ADMIN_PASSWORD:
        request.session["flash"] = "Invalid admin password."
        return RedirectResponse("/admin/login", status_code=status.HTTP_303_SEE_OTHER)
    request.session.clear()
    request.session["role"] = "admin"
    return RedirectResponse("/admin/dashboard", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/admin/dashboard", response_class=HTMLResponse)
def admin_dashboard(request: Request, db: OrmSession = Depends(get_db)):
    require_admin_session(request)
    challenges = []
    for ch in db.query(Challenge).order_by(Challenge.created_at.desc()).all():
        count = db.query(Enrollment).filter(Enrollment.challenge_id==ch.id).count()
        challenges.append({
            "name": ch.name, "metric": ch.metric,
            "start_at": ch.start_at.isoformat().replace("+00:00","Z"),
            "end_at": ch.end_at.isoformat().replace("+00:00","Z"),
            "is_active": ch.is_active, "enroll_count": count
        })
    last = db.query(LeaderboardSnapshot).order_by(LeaderboardSnapshot.run_at.desc()).first()
    last_snapshot = None
    if last:
        payload = json.loads(last.payload)
        # Ensure streak present even for older snapshots & attach scope and challenge_id
        for e in payload.get("entries", []):
            if "streak" not in e:
                u = db.query(User).filter(User.emp_id == e.get("emp_id")).first()
                if u:
                    prefs = get_or_create_user_prefs(db, u.id)
                    s = compute_streaks(db, u.id, prefs.daily_step_goal, lookback_days=120)
                    e["streak"] = s["current_streak_days"]
                else:
                    e["streak"] = 0
        payload["scope"] = last.scope
        payload["challenge_id"] = last.challenge_id
        last_snapshot = payload
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("admin_dashboard.html", {
        "request": request, "title":"Admin Dashboard", "flash": flash,
        "challenges": challenges, "last_snapshot": last_snapshot
    })

@app.get("/admin/challenges/new", response_class=HTMLResponse)
def admin_new_challenge_form(request: Request):
    require_admin_session(request)
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("admin_new_challenge.html", {"request": request, "title":"New Challenge", "flash": flash})

@app.post("/admin/challenges/new")
def admin_new_challenge(request: Request, name: str = Form(...), metric: str = Form(...),
                        start_at: str = Form(...), end_at: str = Form(...), db: OrmSession = Depends(get_db)):
    require_admin_session(request)
    try:
        s = parse_iso_utc(start_at); e = parse_iso_utc(end_at)
        if e <= s: raise ValueError("End must be after start")
    except Exception as ex:
        request.session["flash"] = f"Invalid timestamps: {ex}"
        return RedirectResponse("/admin/challenges/new", status_code=status.HTTP_303_SEE_OTHER)
    if metric not in ("steps","calories"):
        request.session["flash"] = "Metric must be 'steps' or 'calories'."
        return RedirectResponse("/admin/challenges/new", status_code=status.HTTP_303_SEE_OTHER)
    ch = Challenge(name=name.strip(), metric=metric, start_at=s, end_at=e, is_active=True)
    db.add(ch); db.commit()
    request.session["flash"] = "Challenge created."
    return RedirectResponse("/admin/challenges", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/admin/challenges", response_class=HTMLResponse)
def admin_challenges(request: Request, db: OrmSession = Depends(get_db)):
    require_admin_session(request)
    items = db.query(Challenge).order_by(Challenge.created_at.desc()).all()
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("admin_challenges.html", {"request": request, "title":"Challenges", "flash": flash, "items": items})

@app.get("/admin/challenges/{challenge_id}/close")
def admin_close_challenge(request: Request, challenge_id: int, db: OrmSession = Depends(get_db)):
    require_admin_session(request)
    ch = db.query(Challenge).get(challenge_id)
    if not ch:
        request.session["flash"] = "Challenge not found."
        return RedirectResponse("/admin/challenges", status_code=status.HTTP_303_SEE_OTHER)
    entries = compute_leaderboard(db, ch)
    store_snapshot(db, ch.id, "final", entries, ch.metric)
    ch.is_active = False; db.commit()
    request.session["flash"] = "Challenge closed and final leaderboard stored."
    return RedirectResponse("/admin/challenges", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/admin/challenges/{challenge_id}/delete")
def admin_delete_challenge(request: Request, challenge_id: int, db: OrmSession = Depends(get_db)):
    require_admin_session(request)
    ch = db.query(Challenge).get(challenge_id)
    if ch:
        db.query(Enrollment).filter(Enrollment.challenge_id==challenge_id).delete()
        db.query(LeaderboardSnapshot).filter(LeaderboardSnapshot.challenge_id==challenge_id).delete()
        db.delete(ch); db.commit()
        request.session["flash"] = "Challenge deleted."
    else:
        request.session["flash"] = "Challenge not found."
    return RedirectResponse("/admin/challenges", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/admin/hr", response_class=HTMLResponse)
def admin_hr_form(request: Request):
    require_admin_session(request)
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("admin_hr.html", {"request": request, "title":"HR Import", "flash": flash})

@app.post("/admin/hr")
def admin_hr_upload(request: Request, file: UploadFile = File(...), db: OrmSession = Depends(get_db)):
    require_admin_session(request)
    try:
        content = file.file.read().decode("utf-8")
        reader = csv.DictReader(io.StringIO(content))
        created = 0
        for row in reader:
            name = (row.get("name") or "").strip()
            email = (row.get("email") or "").strip()
            emp_id = (row.get("emp_id") or "").strip()
            team_name = (row.get("team") or "").strip()
            if not (name and emp_id):
                continue
            if team_name:
                team = db.query(Team).filter(Team.name==team_name).first()
                if not team:
                    team = Team(name=team_name); db.add(team); db.commit()
            else:
                team = None
            if db.query(User).filter(User.emp_id==emp_id).first():
                continue
            user = User(emp_id=emp_id, name=name, email=email, password_hash=hash_password("Password@123"),
                        is_admin=False, team_id=(team.id if team else None))
            db.add(user); db.commit()
            if not user.prefs:
                db.add(UserPrefs(user_id=user.id, daily_step_goal=8000)); db.commit()
            created += 1
        request.session["flash"] = f"Imported {created} users."
    except Exception as ex:
        request.session["flash"] = f"Import failed: {ex}"
    return RedirectResponse("/admin/hr", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/admin/users", response_class=HTMLResponse)
def admin_users(request: Request, db: OrmSession = Depends(get_db)):
    require_admin_session(request)
    users = db.query(User).order_by(User.created_at.desc()).all()
    data = [{"name":u.name,"emp_id":u.emp_id,"email":u.email,"team":(u.team.name if u.team else None),"is_active":u.is_active} for u in users]
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("admin_users.html", {"request": request, "title":"Users", "flash": flash, "users": data})

@app.get("/admin/users/{emp_id}/disable")
def admin_toggle_user(request: Request, emp_id: str, db: OrmSession = Depends(get_db)):
    require_admin_session(request)
    u = db.query(User).filter(User.emp_id==emp_id).first()
    if u:
        u.is_active = not u.is_active; db.commit()
        request.session["flash"] = f"User {emp_id} active={u.is_active}"
    else:
        request.session["flash"] = "User not found."
    return RedirectResponse("/admin/users", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/admin/users/{emp_id}/reset")
def admin_reset_user_pw(request: Request, emp_id: str, db: OrmSession = Depends(get_db)):
    require_admin_session(request)
    u = db.query(User).filter(User.emp_id==emp_id).first()
    if u:
        u.password_hash = hash_password("Password@123"); db.commit()
        request.session["flash"] = f"Password reset for {emp_id} to 'Password@123'"
    else:
        request.session["flash"] = "User not found."
    return RedirectResponse("/admin/users", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/admin/leaderboard/run")
def admin_run_leaderboard(request: Request):
    require_admin_session(request)
    recompute_all_leaderboards(scope="daily")
    request.session["flash"] = "Leaderboard recomputation kicked off."
    return RedirectResponse("/admin/dashboard", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/admin/leaderboard/export")
def admin_export_leaderboard(request: Request, challenge_id: int = Query(...), scope: str = Query("final"), db: OrmSession = Depends(get_db)):
    require_admin_session(request)
    snap = db.query(LeaderboardSnapshot).filter(
        LeaderboardSnapshot.challenge_id==challenge_id,
        LeaderboardSnapshot.scope==scope
    ).order_by(LeaderboardSnapshot.run_at.desc()).first()
    if not snap:
        raise HTTPException(404, "Snapshot not found.")
    payload = json.loads(snap.payload)
    rows = [["rank","name","emp_id","team","total","streak","metric","run_at"]]
    for e in payload["entries"]:
        rows.append([e.get("rank"), e.get("name"), e.get("emp_id"), e.get("team"), e.get("total"), e.get("streak"), payload["metric"], payload["when"]])
    def gen():
        out = io.StringIO()
        w = csv.writer(out)
        for r in rows:
            w.writerow(r); yield out.getvalue(); out.seek(0); out.truncate(0)
    return StreamingResponse(gen(), media_type="text/csv", headers={"Content-Disposition": f"attachment; filename=leaderboard_{challenge_id}_{scope}.csv"})

# ---------------------------
# Employee
# ---------------------------
@app.get("/employee", response_class=HTMLResponse)
def employee_home(request: Request):
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("employee_home.html", {"request": request, "title":"Employee", "flash": flash})

@app.get("/employee/signup", response_class=HTMLResponse)
def employee_signup_form(request: Request):
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("employee_signup.html", {"request": request, "title":"Sign Up", "flash": flash})

@app.post("/employee/signup")
def employee_signup(
    request: Request,
    name: str = Form(...),
    emp_id: str = Form(...),
    email: str = Form(...),
    password: str = Form(...),
    daily_step_goal: int = Form(...),
    db: OrmSession = Depends(get_db),
):
    emp_id = emp_id.strip(); name = name.strip(); email = email.strip()
    if db.query(User).filter(User.emp_id == emp_id).first():
        request.session["flash"] = f"Employee ID '{emp_id}' already exists. Please try a different ID or sign in."
        return RedirectResponse("/employee/signup", status_code=status.HTTP_303_SEE_OTHER)
    user = User(emp_id=emp_id, name=name, email=email, password_hash=hash_password(password), is_admin=False)
    db.add(user); db.commit()
    db.add(UserPrefs(user_id=user.id, daily_step_goal=max(0, int(daily_step_goal)))); db.commit()
    request.session.clear(); request.session["role"]="employee"; request.session["emp_id"]=emp_id
    request.session["flash"]="Account created successfully. Welcome!"
    return RedirectResponse("/employee/dashboard", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/employee/login", response_class=HTMLResponse)
def employee_login_form(request: Request):
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("employee_login.html", {"request": request, "title":"Sign In", "flash": flash})

@app.post("/employee/login")
def employee_login(request: Request, emp_id: str = Form(...), password: str = Form(...), db: OrmSession = Depends(get_db)):
    emp_id = emp_id.strip()
    user = db.query(User).filter(User.emp_id == emp_id).first()
    if not user or not verify_password(password, user.password_hash):
        request.session["flash"] = "Invalid Employee ID or password."
        return RedirectResponse("/employee/login", status_code=status.HTTP_303_SEE_OTHER)
    if not user.is_active:
        request.session["flash"] = "Account disabled. Contact admin."
        return RedirectResponse("/employee/login", status_code=status.HTTP_303_SEE_OTHER)
    request.session.clear(); request.session["role"]="employee"; request.session["emp_id"]=emp_id
    return RedirectResponse("/employee/dashboard", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/employee/forgot", response_class=HTMLResponse)
def employee_forgot_form(request: Request):
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("employee_forgot.html", {"request": request, "title":"Forgot Password", "flash": flash})

@app.post("/employee/forgot")
def employee_forgot(request: Request, emp_id: str = Form(...), new_password: str = Form(...), code: str = Form(...), db: OrmSession = Depends(get_db)):
    if code != "000000":
        request.session["flash"] = "Invalid verification code (demo uses 000000)."
        return RedirectResponse("/employee/forgot", status_code=status.HTTP_303_SEE_OTHER)
    u = db.query(User).filter(User.emp_id==emp_id).first()
    if not u:
        request.session["flash"]="User not found."
        return RedirectResponse("/employee/forgot", status_code=status.HTTP_303_SEE_OTHER)
    u.password_hash = hash_password(new_password); db.commit()
    request.session["flash"]="Password reset successful. Please sign in."
    return RedirectResponse("/employee/login", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/employee/dashboard", response_class=HTMLResponse)
def employee_dashboard(request: Request, db: OrmSession = Depends(get_db)):
    require_employee_session(request)
    emp_id = request.session.get("emp_id")
    user = db.query(User).filter(User.emp_id == emp_id).first()
    if not user:
        request.session.clear(); request.session["flash"] = "Session expired. Please sign in again."
        return RedirectResponse("/employee/login", status_code=status.HTTP_303_SEE_OTHER)
    tday = today_utc()
    today_row = db.query(ActivityLog).filter(ActivityLog.user_id==user.id, ActivityLog.date_utc==tday).first()
    today_stats = type("obj", (), {})()
    today_stats.steps = (today_row.steps if today_row and not today_row.flagged else 0)
    today_stats.calories = (today_row.calories if today_row and not today_row.flagged else 0)

    def avg_over(days: int, attr: str) -> int:
        start = tday - timedelta(days=days-1)
        q = db.query(ActivityLog).filter(
            ActivityLog.user_id==user.id,
            ActivityLog.date_utc >= start,
            ActivityLog.date_utc <= tday
        )
        vals = [getattr(r, attr) for r in q if not r.flagged]
        return int(sum(vals)/len(vals)) if vals else 0

    avg_steps_7  = avg_over(7, "steps")
    avg_steps_30 = avg_over(30, "steps")
    avg_cal_7    = avg_over(7, "calories")
    avg_cal_30   = avg_over(30, "calories")

    prefs = get_or_create_user_prefs(db, user.id)
    streaks = compute_streaks(db, user.id, prefs.daily_step_goal, lookback_days=120)

    personal = {
        "goal": prefs.daily_step_goal,
        "avg_steps_7": avg_steps_7,
        "avg_steps_30": avg_steps_30,
        "avg_cal_7": avg_cal_7,
        "avg_cal_30": avg_cal_30,
        "current_streak": streaks["current_streak_days"],
        "longest_streak": streaks["longest_streak_days"],
    }

    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("employee_dashboard.html", {
        "request": request, "title": "My Dashboard", "flash": flash, "user": user,
        "today": tday.isoformat(), "today_stats": today_stats, "personal": personal
    })

@app.get("/employee/goals", response_class=HTMLResponse)
def employee_goals_form(request: Request, db: OrmSession = Depends(get_db)):
    require_employee_session(request)
    emp_id = request.session.get("emp_id")
    user = db.query(User).filter(User.emp_id == emp_id).first()
    prefs = get_or_create_user_prefs(db, user.id)
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("employee_goals.html", {
        "request": request, "title":"My Goal", "flash": flash, "daily_step_goal": prefs.daily_step_goal
    })

@app.post("/employee/goals")
def employee_goals_save(request: Request, daily_step_goal: int = Form(...), db: OrmSession = Depends(get_db)):
    require_employee_session(request)
    emp_id = request.session.get("emp_id")
    user = db.query(User).filter(User.emp_id == emp_id).first()
    prefs = get_or_create_user_prefs(db, user.id)
    prefs.daily_step_goal = max(0, int(daily_step_goal))
    db.commit()
    request.session["flash"] = "Goal updated."
    return RedirectResponse("/employee/dashboard", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/employee/wearable", response_class=HTMLResponse)
def employee_wearable_form(request: Request):
    require_employee_session(request)
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("employee_wearable.html", {"request": request, "title":"Wearable", "flash": flash})

@app.post("/employee/wearable")
def employee_wearable_save(request: Request, provider: str = Form(...), token: str = Form("demo-token"), db: OrmSession = Depends(get_db)):
    require_employee_session(request)
    emp_id = request.session.get("emp_id")
    user = db.query(User).filter(User.emp_id == emp_id).first()
    user.wearable_provider = provider; user.wearable_token = token; db.commit()
    request.session["flash"] = "Wearable connected (mock)."
    return RedirectResponse("/employee/dashboard", status_code=status.HTTP_303_SEE_OTHER)

@app.get("/employee/challenges", response_class=HTMLResponse)
def employee_challenges(request: Request, db: OrmSession = Depends(get_db)):
    require_employee_session(request)
    emp_id = request.session.get("emp_id"); user = db.query(User).filter(User.emp_id==emp_id).first()
    now = utcnow()
    # SHOW challenges open for enrollment (active & not ended) — includes upcoming + running
    open_ch = db.query(Challenge).filter(
        Challenge.end_at >= now,
        Challenge.is_active == True
    ).all()
    my_ids = [e.challenge_id for e in db.query(Enrollment).filter(Enrollment.user_id==user.id).all()]
    my_ch = db.query(Challenge).filter(Challenge.id.in_(my_ids)).all() if my_ids else []
    flash = request.session.pop("flash", None)
    return templates.TemplateResponse("employee_challenges.html", {"request": request, "title":"Challenges", "flash": flash,
        "open_challenges": open_ch, "my_challenges": my_ch })

@app.get("/employee/challenges/enroll")
def employee_enroll(request: Request, challenge_id: int, db: OrmSession = Depends(get_db)):
    require_employee_session(request)
    emp_id = request.session.get("emp_id"); user = db.query(User).filter(User.emp_id==emp_id).first()
    ch = db.query(Challenge).get(challenge_id)
    if not ch or not enrollment_allowed(ch):
        request.session["flash"]="Challenge not open."; return RedirectResponse("/employee/challenges", status_code=status.HTTP_303_SEE_OTHER)
    # allow concurrent enrollments; prevent duplicate for same challenge
    if not db.query(Enrollment).filter(Enrollment.user_id==user.id, Enrollment.challenge_id==ch.id).first():
        db.add(Enrollment(user_id=user.id, challenge_id=ch.id)); db.commit()
        request.session["flash"]="Enrolled."
    else:
        request.session["flash"]="Already enrolled."
    return RedirectResponse("/employee/challenges", status_code=status.HTTP_303_SEE_OTHER)

# ---------------------------
# JSON API v1
# ---------------------------
@app.get("/api/v1/challenges")
def api_list_challenges(db: OrmSession = Depends(get_db)):
    items = db.query(Challenge).order_by(Challenge.created_at.desc()).all()
    return {"challenges":[{
        "id":c.id, "name":c.name, "metric":c.metric,
        "start_at":c.start_at.isoformat().replace("+00:00","Z"),
        "end_at":c.end_at.isoformat().replace("+00:00","Z"),
        "is_active":c.is_active
    } for c in items]}

@app.get("/api/v1/challenges/open")
def api_open_challenges(db: OrmSession = Depends(get_db)):
    now = utcnow()
    items = db.query(Challenge).filter(
        Challenge.end_at >= now,
        Challenge.is_active == True
    ).order_by(Challenge.start_at.asc()).all()
    return {"challenges": [{
        "id": c.id,
        "name": c.name,
        "metric": c.metric,
        "start_at": c.start_at.isoformat().replace("+00:00","Z"),
        "end_at": c.end_at.isoformat().replace("+00:00","Z"),
        "is_active": c.is_active
    } for c in items]}

@app.post("/api/v1/auth/login")
def api_login(payload: Dict[str, Any], db: OrmSession = Depends(get_db)):
    emp_id = (payload.get("emp_id") or "").strip()
    password = payload.get("password") or ""
    u = db.query(User).filter(User.emp_id == emp_id).first()
    if not u or not verify_password(password, u.password_hash) or not u.is_active:
        raise HTTPException(status_code=401, detail="Invalid credentials")
    prefs = get_or_create_user_prefs(db, u.id)
    return {
        "emp_id": u.emp_id,
        "name": u.name,
        "email": u.email,
        "team": (u.team.name if u.team else None),
        "daily_step_goal": prefs.daily_step_goal,
    }

@app.post("/api/v1/enroll")
def api_enroll(payload: Dict[str, Any], db: OrmSession = Depends(get_db)):
    emp_id = (payload.get("emp_id") or "").strip()
    password = payload.get("password") or ""
    challenge_id = int(payload.get("challenge_id") or 0)
    u = db.query(User).filter(User.emp_id == emp_id).first()
    if not u or not verify_password(password, u.password_hash) or not u.is_active:
        raise HTTPException(status_code=401, detail="Invalid credentials")
    ch = db.query(Challenge).get(challenge_id)
    if not ch:
        raise HTTPException(status_code=404, detail="Challenge not found")
    if not (ch.is_active and utcnow() <= ch.end_at):
        raise HTTPException(status_code=400, detail="Challenge not open for enrollment")
    exists = db.query(Enrollment).filter(Enrollment.user_id==u.id, Enrollment.challenge_id==ch.id).first()
    if not exists:
        db.add(Enrollment(user_id=u.id, challenge_id=ch.id)); db.commit()
    return {"status":"ok","message":"Enrolled"}

@app.get("/api/v1/user/summary")
def api_user_summary(emp_id: str = Query(...), password: str = Query(...), db: OrmSession = Depends(get_db)):
    u = db.query(User).filter(User.emp_id == emp_id).first()
    if not u or not verify_password(password, u.password_hash) or not u.is_active:
        raise HTTPException(status_code=401, detail="Invalid credentials")
    tday = today_utc()
    row = db.query(ActivityLog).filter(ActivityLog.user_id==u.id, ActivityLog.date_utc==tday).first()
    steps = (row.steps if row and not row.flagged else 0)
    calories = (row.calories if row and not row.flagged else 0)
    prefs = get_or_create_user_prefs(db, u.id)
    streaks = compute_streaks(db, u.id, prefs.daily_step_goal, lookback_days=120)
    return {
        "emp_id": u.emp_id,
        "name": u.name,
        "team": (u.team.name if u.team else None),
        "today": tday.isoformat(),
        "today_steps": steps,
        "today_calories": calories,
        "daily_step_goal": prefs.daily_step_goal,
        "current_streak": streaks["current_streak_days"],
        "longest_streak": streaks["longest_streak_days"],
    }

@app.get("/api/v1/leaderboard")
def api_leaderboard(challenge_id: int = Query(...), scope: str = Query("current"), db: OrmSession = Depends(get_db)):
    ch = db.query(Challenge).get(challenge_id)
    if not ch: raise HTTPException(404, "Challenge not found.")
    if scope == "current":
        entries = compute_leaderboard(db, ch)
        payload = {"metric": ch.metric, "entries": entries, "when": utcnow().isoformat().replace("+00:00","Z")}
        return payload
    else:
        snap = db.query(LeaderboardSnapshot).filter(
            LeaderboardSnapshot.challenge_id==challenge_id,
            LeaderboardSnapshot.scope==scope
        ).order_by(LeaderboardSnapshot.run_at.desc()).first()
        if not snap: raise HTTPException(404, "Snapshot not found.")
        return json.loads(snap.payload)

@app.get("/api/v1/leaderboard/team")
def api_leaderboard_team(challenge_id: int = Query(...), db: OrmSession = Depends(get_db)):
    ch = db.query(Challenge).get(challenge_id)
    if not ch: raise HTTPException(404, "Challenge not found.")
    entries = compute_leaderboard(db, ch)
    team_totals: Dict[str,int] = {}
    for e in entries:
        team = e.get("team") or "—"
        team_totals[team] = team_totals.get(team, 0) + e["total"]
    rows = [{"team":k, "total":v} for k,v in team_totals.items()]
    rows.sort(key=lambda r: -r["total"])
    for i,r in enumerate(rows, start=1): r["rank"]=i
    return {"metric": ch.metric, "entries": rows, "when": utcnow().isoformat().replace("+00:00","Z")}

@app.post("/api/v1/steps/bulk")
def api_steps_bulk(payload: Dict[str, Any], db: OrmSession = Depends(get_db)):
    """
    payload = {
      "source": "wearable_mock|manual",
      "items": [
         {"emp_id":"EMP1","date":"2025-08-25","steps":12345,"calories":450},
         ...
      ]
    }
    Rule: latest-wins per (user, day). Anti-cheat flags auto-applied; flagged days excluded from totals.
    """
    source = (payload.get("source") or "manual").strip()
    items = payload.get("items") or []
    updated, created = 0, 0
    for it in items:
        emp_id = (it.get("emp_id") or "").strip()
        u = db.query(User).filter(User.emp_id==emp_id).first()
        if not u: continue
        try:
            d = date.fromisoformat(it.get("date"))
        except:
            continue
        steps, calories = clamp_activity(it.get("steps"), it.get("calories"))
        row = db.query(ActivityLog).filter(ActivityLog.user_id==u.id, ActivityLog.date_utc==d).first()
        flagged, reason = anti_cheat_flag(db, u.id, d, steps)
        if row:
            row.steps = steps; row.calories = calories; row.source = source
            row.flagged = flagged; row.flag_reason = (reason or None)
            row.updated_at = utcnow()
            db.commit(); updated += 1
        else:
            row = ActivityLog(user_id=u.id, date_utc=d, steps=steps, calories=calories, source=source,
                              flagged=flagged, flag_reason=(reason or None), created_at=utcnow(), updated_at=utcnow())
            db.add(row); db.commit(); created += 1
    return {"status":"ok","created":created,"updated":updated}

# ---------------------------
# Scheduler (09:29 IST daily)
# ---------------------------
scheduler = BackgroundScheduler(timezone=IST)
scheduler.add_job(lambda: recompute_all_leaderboards(scope="daily"),
                  CronTrigger(hour=9, minute=29, timezone=IST))
scheduler.start()

@app.on_event("startup")
def on_startup():
    # catch-up once on boot
    recompute_all_leaderboards(scope="daily")

# ---------------------------
# Misc routes (logout)
# ---------------------------
@app.get("/logout")
def logout(request: Request):
    request.session.clear()
    request.session["flash"] = "You have been logged out."
    return RedirectResponse("/", status_code=status.HTTP_303_SEE_OTHER)

# ---------------------------
# Run (Robust Cloudflare Tunnel for Colab)
# ---------------------------
def start_cloudflared_safe():
    """
    Starts FastAPI (uvicorn) on an available port and exposes it with Cloudflare Tunnel.
    - Auto-detects free port (prefers 8000)
    - If server already running, reuses that port
    - Installs cloudflared by downloading the static binary (no apt-get)
    - Prints the public trycloudflare.com URL
    """
    def is_port_in_use(port: int) -> bool:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(0.2)
            return s.connect_ex(("127.0.0.1", port)) == 0

    def get_free_port(preferred=8000):
        if not is_port_in_use(preferred):
            return preferred
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(("127.0.0.1", 0))
            return s.getsockname()[1]

    # 1) Pick a port
    port = get_free_port(8000)

    # 2) Ensure uvicorn
    try:
        import uvicorn
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uvicorn"])
        import uvicorn

    # 3) Start server only if not already running on that port
    if not is_port_in_use(port):
        def _run_server():
            uvicorn.run(app, host="0.0.0.0", port=port, log_level="info")
        th = threading.Thread(target=_run_server, daemon=True)
        th.start()
        time.sleep(2)

    if not is_port_in_use(port):
        print(f"❌ Uvicorn didn't start on port {port}. Check logs above.")
        return

    # 4) Ensure cloudflared binary (download static release)
    def ensure_cloudflared():
        target = "/usr/local/bin/cloudflared"
        if shutil.which("cloudflared"):
            return shutil.which("cloudflared")
        url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"
        try:
            subprocess.check_call(["wget", "-q", url, "-O", target])
            os.chmod(target, os.stat(target).st_mode | stat.S_IEXEC)
            return target
        except Exception as e:
            print("❌ Failed to fetch cloudflared binary:", e)
            return None

    cfd = ensure_cloudflared()
    if not cfd:
        print("Falling back tip: you can also use LocalTunnel:\n  !npm install -g localtunnel\n  !lt --port", port)
        return

    # 5) Start tunnel & parse URL
    print(f"✅ Server is live on localhost:{port}. Opening Cloudflare Tunnel…")
    proc = subprocess.Popen(
        [cfd, "tunnel", "--url", f"http://localhost:{port}", "--loglevel", "info"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )

    public_url = None
    for line in iter(proc.stdout.readline, ""):
        if "trycloudflare.com" in line:
            m = re.search(r"https://[^\s]*trycloudflare\.com", line)
            if m:
                public_url = m.group(0)
                print("\n🌐 Public URL:", public_url)
                print("Open that URL and you're live! 🎉")
                break
        if "inf" in line.lower() and "metrics" not in line.lower():
            print(".", end="")

    if not public_url:
        print("\n⚠️ Could not detect the public URL. Scroll logs above for cloudflared output.")

# --- In Colab, run this in a new cell to start the app + tunnel ---
# start_cloudflared_safe()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 17.3 MB/s eta 0:00:00


/tmp/ipython-input-3065296264.py:1327: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [2]:
start_cloudflared_safe()

INFO:     Started server process [147]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


✅ Server is live on localhost:8000. Opening Cloudflare Tunnel…
....
🌐 Public URL: https://race-licence-immediate-philosophy.trycloudflare.com
Open that URL and you're live! 🎉
